In [5]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(URL(
            user='suckFlake',
            password='suckFlake123!',
            account='hn51231.east-us-2.azure',
            database = 'SNOWFLAKE_SAMPLE_DATA',
            schema = 'TPCDS_SF10TCL',
            warehouse = 'COMPUTE_WH',
            role='accountadmin',
            numpy = True
    ))


In [6]:
pd.read_sql_query('select * from WEB_SALES limit 10',engine)

,ws_sold_date_sk,ws_sold_time_sk,ws_ship_date_sk,ws_item_sk,ws_bill_customer_sk,ws_bill_cdemo_sk,ws_bill_hdemo_sk,ws_bill_addr_sk,ws_ship_customer_sk,ws_ship_cdemo_sk,...,ws_ext_wholesale_cost,ws_ext_list_price,ws_ext_tax,ws_coupon_amt,ws_ext_ship_cost,ws_net_paid,ws_net_paid_inc_tax,ws_net_paid_inc_ship,ws_net_paid_inc_ship_tax,ws_net_profit
0,2451773,72580,2451874,65486,9195753,830581,5856,21156684,46407928,122467,...,5590.80,7603.20,333.00,0.00,303.60,5550.00,5883.00,5853.60,6186.60,-40.80
1,2451773,47084,2451874,65486,7126190,280006,669,19262485,41653929,1839062,...,9053.76,20551.68,0.00,0.00,5959.68,10892.16,10892.16,16851.84,16851.84,1838.40
2,2451773,79525,2451826,65486,18569741,783908,1722,4752336,63433045,853298,...,2755.76,3306.76,124.27,0.00,1190.16,1553.44,1677.71,2743.60,2867.87,-1202.32
3,2451773,25375,2451827,65486,46128079,83239,7199,1338359,31814337,869495,...,471.90,967.30,17.79,0.00,125.70,444.90,462.69,570.60,588.39,-27.00
4,2451773,70983,2451866,65486,16861317,1841307,6097,23653287,62235801,196800,...,1538.92,3308.52,224.62,0.00,98.75,3208.98,3433.60,3307.73,3532.35,1670.06
5,2451773,60974,2451808,65486,42680183,1768441,6299,7061757,49990874,521744,...,3902.60,11239.33,42.25,531.29,2922.21,704.27,746.52,3626.48,3668.73,-3198.33
6,2451773,62757,2451816,65486,44592317,913505,4293,9898281,42230390,1441256,...,7499.88,13424.42,261.74,0.00,5906.28,5234.81,5496.55,11141.09,11402.83,-2265.07
7,2451773,46138,2451782,65486,23269630,1202282,181,19935233,10553920,1737764,...,8973.57,25753.56,1277.37,0.00,256.68,15967.17,17244.54,16223.85,17501.22,6993.60
8,2451773,33826,2451840,65486,28485651,1016124,1444,14482317,10168484,565063,...,3592.44,9160.56,52.20,0.00,1373.76,1740.24,1792.44,3114.00,3166.20,-1852.20
9,2451773,70245,2451777,65486,44962219,182965,4511,20847210,1270150,589615,...,5326.65,11984.74,11.50,2731.81,3954.27,143.78,155.28,4098.05,4109.55,-5182.87


In [3]:
q60_c = pd.read_sql_query('select DISTINCT I_category from item limit 100',engine)

,i_category
0,Sports
1,Children
2,Women
3,None
4,Home
5,Electronics
6,Jewelry
7,Men
8,Shoes
9,Music


In [4]:
# # question 2
# pd.read_sql_query('''
# WITH wscs AS
#   (SELECT sold_date_sk,
#           sales_price
#    FROM
#      (SELECT ws_sold_date_sk sold_date_sk,
#              ws_ext_sales_price sales_price
#       FROM web_sales
#       UNION ALL SELECT cs_sold_date_sk sold_date_sk,
#                        cs_ext_sales_price sales_price
#       FROM catalog_sales) sq1
#       ),
#      wswscs AS
#   (SELECT d_week_seq,
#           sum(CASE
#                   WHEN (d_day_name='Sunday') THEN sales_price
#                   ELSE NULL
#               END) sun_sales,
#           sum(CASE
#                   WHEN (d_day_name='Monday') THEN sales_price
#                   ELSE NULL
#               END) mon_sales,
#           sum(CASE
#                   WHEN (d_day_name='Tuesday') THEN sales_price
#                   ELSE NULL
#               END) tue_sales,
#           sum(CASE
#                   WHEN (d_day_name='Wednesday') THEN sales_price
#                   ELSE NULL
#               END) wed_sales,
#           sum(CASE
#                   WHEN (d_day_name='Thursday') THEN sales_price
#                   ELSE NULL
#               END) thu_sales,
#           sum(CASE
#                   WHEN (d_day_name='Friday') THEN sales_price
#                   ELSE NULL
#               END) fri_sales,
#           sum(CASE
#                   WHEN (d_day_name='Saturday') THEN sales_price
#                   ELSE NULL
#               END) sat_sales
#    FROM wscs,
#         date_dim
#    WHERE d_date_sk = sold_date_sk
#    GROUP BY d_week_seq
#    limit 10000)
# SELECT d_week_seq1,
#        round(sun_sales1/sun_sales2, 2),
#        round(mon_sales1/mon_sales2, 2),
#        round(tue_sales1/tue_sales2, 2),
#        round(wed_sales1/wed_sales2, 2),
#        round(thu_sales1/thu_sales2, 2),
#        round(fri_sales1/fri_sales2, 2),
#        round(sat_sales1/sat_sales2, 2)
# FROM
#   (SELECT wswscs.d_week_seq d_week_seq1,
#           sun_sales sun_sales1,
#           mon_sales mon_sales1,
#           tue_sales tue_sales1,
#           wed_sales wed_sales1,
#           thu_sales thu_sales1,
#           fri_sales fri_sales1,
#           sat_sales sat_sales1
#    FROM wswscs,
#         date_dim
#    WHERE date_dim.d_week_seq = wswscs.d_week_seq
#      AND d_year = 2001) y,
#   (SELECT wswscs.d_week_seq d_week_seq2,
#           sun_sales sun_sales2,
#           mon_sales mon_sales2,
#           tue_sales tue_sales2,
#           wed_sales wed_sales2,
#           thu_sales thu_sales2,
#           fri_sales fri_sales2,
#           sat_sales sat_sales2
#    FROM wswscs,
#         date_dim
#    WHERE date_dim.d_week_seq = wswscs.d_week_seq
#      AND d_year = 2001+1) z
# WHERE d_week_seq1 = d_week_seq2-53
# ORDER BY d_week_seq1
# limit 1000;
# ''',engine)

,d_week_seq1,"ROUND(SUN_SALES1/SUN_SALES2, 2)","ROUND(MON_SALES1/MON_SALES2, 2)","ROUND(TUE_SALES1/TUE_SALES2, 2)","ROUND(WED_SALES1/WED_SALES2, 2)","ROUND(THU_SALES1/THU_SALES2, 2)","ROUND(FRI_SALES1/FRI_SALES2, 2)","ROUND(SAT_SALES1/SAT_SALES2, 2)"
0,5270,3.44,1.82,1.89,1.6,3.45,3.44,3.44
1,5270,3.44,1.82,1.89,1.6,3.45,3.44,3.44
2,5270,3.44,1.82,1.89,1.6,3.45,3.44,3.44
3,5270,3.44,1.82,1.89,1.6,3.45,3.44,3.44
4,5270,3.44,1.82,1.89,1.6,3.45,3.44,3.44
...,...,...,...,...,...,...,...,...
995,5291,1.00,1.00,1.00,1.0,1.00,1.00,1.00
996,5291,1.00,1.00,1.00,1.0,1.00,1.00,1.00
997,5291,1.00,1.00,1.00,1.0,1.00,1.00,1.00
998,5291,1.00,1.00,1.00,1.0,1.00,1.00,1.00


In [11]:
#QUESTION1
pd.read_sql_query('''with customer_total_return as
(select sr_customer_sk as ctr_customer_sk
,sr_store_sk as ctr_store_sk
,sum(SR_RETURN_QUANTITY) as ctr_total_return
from store_returns
,date_dim
where sr_returned_date_sk = d_date_sk
and d_year = 2000
group by sr_customer_sk
,sr_store_sk
limit 500)
 select  c_customer_id
from customer_total_return ctr1
,store
,customer
where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
from customer_total_return ctr2
where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
and s_store_sk = ctr1.ctr_store_sk
and s_state = 'TN'
and ctr1.ctr_customer_sk = c_customer_sk
order by c_customer_id
 limit 5;''',engine)

,c_customer_id
0,AAAAAAAAAAPNILAA
1,AAAAAAAADIJLPNAA
2,AAAAAAAAHIKFOLDA
3,AAAAAAAAKLDKMKDA
4,AAAAAAAAKMOJOPAA


In [36]:
pd.read_sql_query('''USE SCHEMA SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL;''',engine)

,status
0,Statement executed successfully.


In [40]:
#Q4 test

pd.read_sql_query('''SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          c_preferred_cust_flag customer_preferred_cust_flag,
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM (SELECT * FROM customer sample row (10000 rows)) customer,
        (SELECT * FROM store_sales sample row (100000 rows)) store_sales,
        (SELECT * FROM date_dim sample row (100000 rows)) date_dim
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
    LIMIT 300''', engine)

,customer_id,customer_first_name,customer_last_name,customer_preferred_cust_flag,customer_birth_country,customer_login,customer_email_address,dyear,year_total,sale_type
0,AAAAAAAAMIGKOEBA,Helen,Peters,N,POLAND,None,Helen.Peters@43XDRJrYpHKVJvbm.org,1999,NaN,s
1,AAAAAAAAKKMHHJCA,Chantel,Murphy,Y,SWEDEN,None,Chantel.Murphy@byYuF0IJjgTg.org,2000,829.025,s
2,AAAAAAAAICLAJADA,Robert,Johnston,N,LESOTHO,None,Robert.Johnston@91g0r8O99u.org,2002,1220.590,s
3,AAAAAAAACONFIICA,Margaret,Moore,N,SRI LANKA,None,Margaret.Moore@0jkrCj2G0Okjazle83.org,1999,336.340,s
4,AAAAAAAAALEFLADA,Philip,Philips,N,MONACO,None,Philip.Philips@l.edu,1998,243.760,s
5,AAAAAAAAGHJMLBAA,Jerry,Barr,N,CHINA,None,Jerry.Barr@Ij9.edu,1998,2395.745,s
6,AAAAAAAAGGPMHNBA,Nikki,Buford,Y,GUAM,None,Nikki.Buford@XJptcd0.org,1999,693.600,s
7,AAAAAAAAOMIKDNDA,Martin,Ward,N,SAUDI ARABIA,None,Martin.Ward@cxX.org,1998,656.145,s
8,AAAAAAAAENGKAHCA,Jeannine,Dew,N,BAHRAIN,None,Jeannine.Dew@7ao8ECtCTmqZ.org,2001,2049.720,s
9,AAAAAAAAECOMDBDA,Kimberly,Jones,Y,GHANA,None,Kimberly.Jones@7KQDsz.org,2001,1978.470,s


In [28]:
#Question 4

pd.read_sql_query('''WITH year_total AS
  ((SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          c_preferred_cust_flag customer_preferred_cust_flag,
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM (SELECT * FROM customer LIMIT 100) AS customer,
        (SELECT * FROM store_sales LIMIT 100) AS store_sales,
        (SELECT * FROM date_dim LIMIT 100) AS date_dim
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
    LIMIT 300)
   UNION ALL (SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2)) year_total,
                    'c' sale_type
   FROM customer,
        catalog_sales,
        date_dim
   WHERE c_customer_sk = cs_bill_customer_sk
     AND cs_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
    LIMIT 300)
   UNION ALL (SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2)) year_total,
                    'w' sale_type
   FROM customer,
        web_sales,
        date_dim
   WHERE c_customer_sk = ws_bill_customer_sk
     AND ws_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
    LIMIT 300))
SELECT t_s_secyear.customer_id,
       t_s_secyear.customer_first_name,
       t_s_secyear.customer_last_name,
       t_s_secyear.customer_preferred_cust_flag
FROM (SELECT * FROM year_total LIMIT 500) t_s_firstyear,
     (SELECT * FROM year_total LIMIT 300) t_s_secyear,
     (SELECT * FROM year_total LIMIT 700) t_c_firstyear,
     (SELECT * FROM year_total LIMIT 200) t_c_secyear,
     (SELECT * FROM year_total LIMIT 400) t_w_firstyear,
     (SELECT * FROM year_total LIMIT 600) t_w_secyear
WHERE t_s_secyear.customer_id = t_s_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_c_secyear.customer_id
  AND t_s_firstyear.customer_id = t_c_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_secyear.customer_id
  AND t_s_firstyear.sale_type = 's'
  AND t_c_firstyear.sale_type = 'c'
  AND t_w_firstyear.sale_type = 'w'
  AND t_s_secyear.sale_type = 's'
  AND t_c_secyear.sale_type = 'c'
  AND t_w_secyear.sale_type = 'w'
  AND t_s_firstyear.dyear = 2001
  AND t_s_secyear.dyear = 2001+1
  AND t_c_firstyear.dyear = 2001
  AND t_c_secyear.dyear = 2001+1
  AND t_w_firstyear.dyear = 2001
  AND t_w_secyear.dyear = 2001+1
  AND t_s_firstyear.year_total > 0
  AND t_c_firstyear.year_total > 0
  AND t_w_firstyear.year_total > 0
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_s_firstyear.year_total > 0 THEN t_s_secyear.year_total / t_s_firstyear.year_total
                ELSE NULL
            END
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_w_firstyear.year_total > 0 THEN t_w_secyear.year_total / t_w_firstyear.year_total
                ELSE NULL
            END
ORDER BY t_s_secyear.customer_id,
         t_s_secyear.customer_first_name,
         t_s_secyear.customer_last_name,
         t_s_secyear.customer_preferred_cust_flag
LIMIT 50;''',engine)




ProgrammingError: (snowflake.connector.errors.ProgrammingError) 002003 (42S02): SQL compilation error:
Object 'CUSTOMER' does not exist or not authorized.
[SQL: WITH year_total AS
  ((SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          c_preferred_cust_flag customer_preferred_cust_flag,
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM (SELECT * FROM customer LIMIT 100) AS customer,
        (SELECT * FROM store_sales LIMIT 100) AS store_sales,
        (SELECT * FROM date_dim LIMIT 100) AS date_dim
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            

In [6]:
#Add variable in json here!

import json
var_store = {}
var_store['q1_state'] = ['TN',
 'NM',
 'KY',
 'IA',
 'NC',
 'OH',
 'GA',
 'IN',
 'AL',
 'SC',
 'MD',
 'LA',
 'SD',
 'MI',
 'FL',
 'WV',
 'VT',
 'TX',
 'MT',
 'CA',
 'NJ',
 'CO',
 'OK',
 'IL',
 'None',
 'NE',
 'OR',
 'NY',
 'MO',
 'WI',
 'VA',
 'PA',
 'WA',
 'KS',
 'MN']



var_store['q1_agg'] = ['SR_RETURN_AMT','SR_RETURN_AMT_INC_TAX','SR_RETURN_QUANTITY','SR_RETURN_TAX','SR_RETURN_SHIP_COST']
var_store['q3_aggc']= ["ss_sold_date_sk","ss_sold_time_sk","ss_item_sk","ss_customer_sk","ss_cdemo_sk","ss_hdemo_sk","ss_addr_sk","ss_store_sk","ss_promo_sk","ss_ticket_number","ss_quantity","ss_wholesale_cost","ss_list_price","ss_sales_price","ss_ext_discount_amt","ss_ext_sales_price","ss_ext_wholesale_cost","ss_ext_list_price","ss_ext_tax","ss_coupon_amt","ss_net_paid","ss_net_paid_inc_tax","ss_net_profit"] 
var_store['q60_c'] = list(q60_c.values.T[0])
var_store

var_store_json = json.dumps(var_store,sort_keys=False,separators=(',', ': '))
# Writing to var_store.json
with open("var_store.json", "w") as outfile:
    outfile.write(var_store_json)

In [7]:
type(json.load(open('var_store.json'))['q1_state'])
type(json.load(open('var_store.json'))['q3_aggc'])
type(json.load(open('var_store.json'))['q60_c'])


list

In [15]:
 #Query 4 limit 300 back up
pd.read_sql_query('''WITH year_total AS
  ((SELECT c_customer_id customer_id,
          c_first_name customer_first_name,
          c_last_name customer_last_name,
          c_preferred_cust_flag customer_preferred_cust_flag,
          c_birth_country customer_birth_country,
          c_login customer_login,
          c_email_address customer_email_address,
          d_year dyear,
          sum(((ss_ext_list_price-ss_ext_wholesale_cost-ss_ext_discount_amt)+ss_ext_sales_price)/2) year_total,
          's' sale_type
   FROM (SELECT * FROM customer LIMIT 300),
        (SELECT * FROM store_sales LIMIT 300,
        (SELECT * FROM date_dim LIMIT 300)
   WHERE c_customer_sk = ss_customer_sk
     AND ss_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
    LIMIT 300)
   UNION ALL (SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((cs_ext_list_price-cs_ext_wholesale_cost-cs_ext_discount_amt)+cs_ext_sales_price)/2)) year_total,
                    'c' sale_type
   FROM (SELECT * FROM customer LIMIT 300),
        (SELECT * FROM catalog_sales LIMIT 300),
        (SELECT * FROM date_dim LIMIT 300)
   WHERE c_customer_sk = cs_bill_customer_sk
     AND cs_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
    LIMIT 300)
   UNION ALL (SELECT c_customer_id customer_id,
                    c_first_name customer_first_name,
                    c_last_name customer_last_name,
                    c_preferred_cust_flag customer_preferred_cust_flag,
                    c_birth_country customer_birth_country,
                    c_login customer_login,
                    c_email_address customer_email_address,
                    d_year dyear,
                    sum((((ws_ext_list_price-ws_ext_wholesale_cost-ws_ext_discount_amt)+ws_ext_sales_price)/2)) year_total,
                    'w' sale_type
   FROM customer,
        web_sales,
        date_dim
   WHERE c_customer_sk = ws_bill_customer_sk
     AND ws_sold_date_sk = d_date_sk
   GROUP BY c_customer_id,
            c_first_name,
            c_last_name,
            c_preferred_cust_flag,
            c_birth_country,
            c_login,
            c_email_address,
            d_year
    LIMIT 300))
SELECT t_s_secyear.customer_id,
       t_s_secyear.customer_first_name,
       t_s_secyear.customer_last_name,
       t_s_secyear.customer_preferred_cust_flag
FROM year_total t_s_firstyear,
     year_total t_s_secyear,
     year_total t_c_firstyear,
     year_total t_c_secyear,
     year_total t_w_firstyear,
     year_total t_w_secyear
WHERE t_s_secyear.customer_id = t_s_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_c_secyear.customer_id
  AND t_s_firstyear.customer_id = t_c_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_firstyear.customer_id
  AND t_s_firstyear.customer_id = t_w_secyear.customer_id
  AND t_s_firstyear.sale_type = 's'
  AND t_c_firstyear.sale_type = 'c'
  AND t_w_firstyear.sale_type = 'w'
  AND t_s_secyear.sale_type = 's'
  AND t_c_secyear.sale_type = 'c'
  AND t_w_secyear.sale_type = 'w'
  AND t_s_firstyear.dyear = 2001
  AND t_s_secyear.dyear = 2001+1
  AND t_c_firstyear.dyear = 2001
  AND t_c_secyear.dyear = 2001+1
  AND t_w_firstyear.dyear = 2001
  AND t_w_secyear.dyear = 2001+1
  AND t_s_firstyear.year_total > 0
  AND t_c_firstyear.year_total > 0
  AND t_w_firstyear.year_total > 0
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_s_firstyear.year_total > 0 THEN t_s_secyear.year_total / t_s_firstyear.year_total
                ELSE NULL
            END
  AND CASE
          WHEN t_c_firstyear.year_total > 0 THEN t_c_secyear.year_total / t_c_firstyear.year_total
          ELSE NULL
      END > CASE
                WHEN t_w_firstyear.year_total > 0 THEN t_w_secyear.year_total / t_w_firstyear.year_total
                ELSE NULL
            END
ORDER BY t_s_secyear.customer_id,
         t_s_secyear.customer_first_name,
         t_s_secyear.customer_last_name,
         t_s_secyear.customer_preferred_cust_flag
LIMIT 50;''',engine)





ProgrammingError: (snowflake.connector.errors.ProgrammingError) 001003 (42000): SQL compilation error:
syntax error line 2 at position 3 unexpected 'SELECT'.
syntax error line 3 at position 22 unexpected 'customer_first_name'.
syntax error line 4 at position 21 unexpected 'customer_last_name'.
syntax error line 5 at position 31 unexpected 'customer_preferred_cust_flag'.
syntax error line 6 at position 25 unexpected 'customer_birth_country'.
syntax error line 7 at position 17 unexpected 'customer_login'.
syntax error line 8 at position 25 unexpected 'customer_email_address'.
syntax error line 9 at position 16 unexpected 'dyear'.
syntax error line 10 at position 13 unexpected '('.
syntax error line 10 at position 75 unexpected '+'.
[SQL: WITH year_total AS
 ((SELECT c_customer_id customer_id,
         c_first_name customer_first_name,
         c_last_name customer_last_name,
         c_preferred_cust_flag customer_preferred_cust_flag,
         c_birth_country customer_birth_country,
    

In [2]:
pd.read_sql_query('select max(s_gmt_offset ), min(s_gmt_offset ) from store ',engine)

,MAX(S_GMT_OFFSET ),MIN(S_GMT_OFFSET )
0,-5.0,-8.0
